In [37]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
def GetLatest():
  url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
  parameters = {
    'start':'1',
    'limit':'5000',
    'convert':'USD'
  }
  headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'c0fef76c-7d4e-43e1-bdfa-3b02bead65e0',
  }

  session = Session()
  session.headers.update(headers)

  try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
  return data

In [38]:
def GetMap(id):
  url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/category'
  parameters = {
    'id':f'{id}',
  }
  headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'c0fef76c-7d4e-43e1-bdfa-3b02bead65e0',
  }

  session = Session()
  session.headers.update(headers)

  try:
    response = session.get(url, params=parameters)
    map = json.loads(response.text)
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
  return map

In [39]:
def GetCategories():
  url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/categories'
  parameters = {}
  headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'c0fef76c-7d4e-43e1-bdfa-3b02bead65e0',
  }

  session = Session()
  session.headers.update(headers)

  try:
    response = session.get(url, params=parameters)
    categories = json.loads(response.text)
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
  return categories

In [36]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()
# import datetime


# def Streaming():
#     DATA = []
#     categories = GetCategories()
#     Dict = {
#         "CategoryName" :None,"CoinName" :None,"NumMarketPairs" :None,"DateAdded" :None,"IsActive" :None,"CMCRank" :None,"PriceUSD" :None,
#         "Volume24" :None,"PercentChange1h" :None,"PercentChange24h" :None,"PercentChange7d" :None,"MarketCap" :None,"LastUpdated" :None,"Time":None
#         }
#     for i in range(len(categories['data'])):
#         ID = categories['data'][i]['id']
#         map = GetMap(ID)
#         try:
#             for j in range(len(map["data"]["coins"])):
#                 Dict["CategoryName"] = categories['data'][j]['name']
#                 Dict["CoinName"] = map['data']['coins'][0]['name']
#                 Dict["NumMarketPairs"] = map['data']['coins'][0]['num_market_pairs']
#                 Dict["DateAdded"] = map['data']['coins'][0]['date_added']
#                 Dict["IsActive"] = map['data']['coins'][0]['is_active']
#                 Dict["CMCRank"] = map['data']['coins'][0]['cmc_rank']
#                 Dict["PriceUSD"] = map['data']['coins'][0]["quote"]["USD"]['price']
#                 Dict["Volume24"] = map['data']['coins'][0]["quote"]["USD"]['volume_24h']
#                 Dict["PercentChange1h"] = map['data']['coins'][0]["quote"]["USD"]['percent_change_1h']
#                 Dict["PercentChange24h"] = map['data']['coins'][0]["quote"]["USD"]['percent_change_24h']
#                 Dict["PercentChange7d"] =  map['data']['coins'][0]["quote"]["USD"]['percent_change_7d']
#                 Dict["MarketCap"] = map['data']['coins'][0]["quote"]["USD"]['market_cap']
#                 Dict["LastUpdated"] = map['data']['coins'][0]["quote"]["USD"]['last_updated']
#                 Dict["Time"] = str(datetime.datetime.now())
#                 DATA.append(Dict)
#                 df = spark.createDataFrame(DATA)
#                 df.show()
#         except Exception as e:
#             print(e)
#             continue  

In [45]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()
import datetime
import json
df = spark.read.json(r'C:\Users\LENOVO\Desktop\Hamza Bouajila\3IDSD SD\Spark\TP\Projet\WeatherForcast\app\data\data.json')

In [46]:
df.groupby("CategoryName").mean().groupby("CategoryName").mean().count()

25

In [47]:
df.printSchema()

root
 |-- CMCRank: double (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- CoinName: string (nullable = true)
 |-- DateAdded: string (nullable = true)
 |-- IsActive: long (nullable = true)
 |-- LastUpdated: string (nullable = true)
 |-- MarketCap: double (nullable = true)
 |-- NumMarketPairs: long (nullable = true)
 |-- PercentChange1h: double (nullable = true)
 |-- PercentChange24h: double (nullable = true)
 |-- PercentChange7d: double (nullable = true)
 |-- PriceUSD: double (nullable = true)
 |-- RealTime: string (nullable = true)
 |-- Volume24: double (nullable = true)



In [48]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly as py

In [49]:
df.show()

+-------+--------------------+--------+--------------------+--------+--------------------+--------------------+--------------+---------------+----------------+---------------+------------------+--------------------+--------------------+
|CMCRank|        CategoryName|CoinName|           DateAdded|IsActive|         LastUpdated|           MarketCap|NumMarketPairs|PercentChange1h|PercentChange24h|PercentChange7d|          PriceUSD|            RealTime|            Volume24|
+-------+--------------------+--------+--------------------+--------+--------------------+--------------------+--------------+---------------+----------------+---------------+------------------+--------------------+--------------------+
|   31.0|eGirl Capital Por...|Lido DAO|2020-12-15T00:00:...|       1|2023-11-23T13:33:...| 2.283347148078477E9|           263|    -1.09020988|      4.33277997|     7.83834381| 2.565003224345331|2023-11-23 14:34:...|1.0934260386819059E8|
|   31.0|Standard Crypto P...|Lido DAO|2020-12-15T00

In [65]:
df.groupby(["CategoryName"]).mean().sort("avg(PriceUSD)",ascending=False)

20